In [ ]:
import numpy as np
import os
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from pyomo.core import * 

f_demand = Path("2050_Data/Consumption_2050.csv")
f_area_price = Path("2050_Data/Border_prices_2050.csv")
f_NTC = Path("2024_Data/NTC_BaseCase.csv")                                           # Scenario-specific transfer capacity for base case scenario
f_inflow = Path("2050_Data/Inflow_2050.csv")
f_k_energy = Path("2050_Data/K_energy_2050.csv")
f_k_prices = Path("2050_Data/K_prices_2050.csv")
f_loadshed = Path("2050_Data/Loadshedding_2050.csv")
f_elkjel = Path("2050_Data/Elkjel_2050.csv")
f_hydro = Path("2050_Data/Hydro_info_total_2050.csv")
f_energieq = Path("2050_Data/Energieq_2050.csv")
f_up_req = Path("2050_Data/Final_up_reserve_BaseCase.csv")                             # Scenario-specific capacity reserve requirements for up-regulation
f_down_req = Path("2050_Data/Final_down_reserve_BaseCase.csv")                         # Scenario-specific capacity reserve requirements for down-regulation

f_NO1_int = Path("2050_Intermittent/NO1_intermittent_energy.csv")
f_NO2_int = Path("2050_Intermittent/NO2_intermittent_energy.csv")
f_NO3_int = Path("2050_Intermittent/NO3_intermittent_energy.csv")
f_NO4_int = Path("2050_Intermittent/NO4_intermittent_energy.csv")
f_NO5_int = Path("2050_Intermittent/NO5_intermittent_energy.csv")
f_SE1_int = Path("2050_Intermittent/SE1_intermittent_energy.csv")
f_SE2_int = Path("2050_Intermittent/SE2_intermittent_energy.csv")
f_SE3_int = Path("2050_Intermittent/SE3_intermittent_energy.csv")
f_SE4_int = Path("2050_Intermittent/SE4_intermittent_energy.csv")


df_demand = pd.read_csv(f_demand, index_col="Timestamp")
df_area_price = pd.read_csv(f_area_price, index_col="Timestamp")
df_NTC = pd.read_csv(f_NTC, index_col="Area")
df_inflow = pd.read_csv(f_inflow, index_col="Timestamp")
df_k_energy = pd.read_csv(f_k_energy, index_col="Area")
df_k_prices = pd.read_csv(f_k_prices, index_col="Timestamp")
df_up_req = pd.read_csv(f_up_req, index_col="Timestamp")
df_down_req = pd.read_csv(f_down_req, index_col="Timestamp")
df_loadshed = pd.read_csv(f_loadshed, index_col="l")
df_loadshed.columns = df_loadshed.columns.str.strip()
df_hydro = pd.read_csv(f_hydro, index_col="Area")
df_elkjel = pd.read_csv(f_elkjel, index_col="Area")
df_energieq = pd.read_csv(f_energieq, index_col="s")

df_int = {
    "NO1": pd.read_csv(f_NO1_int, index_col="Timestamp"),
    "NO2": pd.read_csv(f_NO2_int, index_col="Timestamp"),
    "NO3": pd.read_csv(f_NO3_int, index_col="Timestamp"),
    "NO4": pd.read_csv(f_NO4_int, index_col="Timestamp"),
    "NO5": pd.read_csv(f_NO5_int, index_col="Timestamp"),
    "SE1": pd.read_csv(f_SE1_int, index_col="Timestamp"),
    "SE2": pd.read_csv(f_SE2_int, index_col="Timestamp"),
    "SE3": pd.read_csv(f_SE3_int, index_col="Timestamp"),
    "SE4": pd.read_csv(f_SE4_int, index_col="Timestamp"),
}


step = 1
T = list(df_demand.index.to_numpy()[::step])
N = df_demand.columns.tolist()
J =  df_int["NO1"].columns.tolist()
M = df_NTC.columns.tolist()
K = df_k_energy.columns.tolist()
S = df_energieq.index.tolist()

X_t_n_j = {(t,n,j): df_int[n].at[t, j] for n in N for j in J for t in T}                                    # Produksjon fra intermittent
X_cap_n_k = {(n,k): df_k_energy.loc[n,k] for n in N for k in K}                                             # Installert effekt for k energikilde

X_cap_n_i = {n: df_hydro.loc[n, "Installert effekt [MW]"] for n in N}                                       # Installert effekt
W_start_n_i = {n: df_hydro.loc[n, "Start nivaa [MW]"] for n in N}                                           # Start nivå reservoir
W_cap_n_i = {n: df_hydro.loc[n, "Maks magasinfylling [MWh]"] for n in N}                                    # Maks magasinnivbå
W_inflow_t_n_i = {(t, n): df_inflow.loc[t, n]  for t in T for n in N}                                       # Inflow til hver generator i hvert område

Elkjel_cap = {n: df_elkjel.loc[n, "Installert effekt [MW]"] for n in N}                                     # Installed capacity for electric water heaters 

D_t_n = {(t,n): df_demand.loc[t,n] for t in T for n in N }                                                  # Demand til hvert område

Q_capacity = {                                                                                              # Net transfer capacity for mxm areas 
    (i, j): df_NTC.loc[i, j]
    for i in M
    for j in M
}

C_t_m = {(t, m): df_area_price.loc[t, m] for t in T for m in M}                                             # Prices for neighbouring areas 
C_t_k = {(t,k): df_k_prices.loc[t,k] for t in T for k in K}                                                 # Prices for k energy sources

R_up_t_n = {(t,n): df_up_req.loc[t,n] for t in T for n in N}                                                # Reserve req for up regulation in each area
R_down_t_n = {(t,n): df_down_req.loc[t,n] for t in T for n in N}                                            # Reserve req for down regulation in each area                                                
                                                                    

" Data for energy eqivalent: "

A = df_energieq["A"].to_dict()                                                                              # Alfa in %
B = df_energieq["eeq"].to_dict()                                                                            # Eeq in %

                                                     
"Hardkoding for loadshed:"

L_max_l1 = 200
L_max_l2 = 200
L_max_l3 = 200
L_max_l4 = 200
L_max_l5 = 3200

C_shed_l1 = 120
C_shed_l2 = 300
C_shed_l3 = 750
C_shed_l4 = 1900
C_shed_l5 = 4970   

C_slack_up = 1000
C_slack_down = 1000
C_slack_energy = 1000

In [ ]:
# ---- Create pyomo model, and defining sets, parameters and variables ----- 

model = pyo.ConcreteModel()

"-------- SETS -----------------------"
model.t = pyo.Set(initialize=T)
model.j = pyo.Set(initialize=J)
model.n = pyo.Set(initialize=N)
model.m = pyo.Set(initialize=M)
model.k = pyo.Set(initialize=K)
model.s = pyo.Set(initialize=S)

"------- PARAMETERS ------------------"
model.X_t_n_j = pyo.Param(model.t, model.n, model.j, initialize = X_t_n_j)
model.X_cap_n_i = pyo.Param(model.n, initialize = X_cap_n_i)
model.X_cap_n_k = pyo.Param(model.n, model.k, initialize = X_cap_n_k)
model.W_start_n_i = pyo.Param(model.n, initialize = W_start_n_i)
model.W_cap_n_i = pyo.Param(model.n, initialize = W_cap_n_i)
model.W_inflow_t_n_i = pyo.Param(model.t, model.n, initialize = W_inflow_t_n_i)
model.D_t_n = pyo.Param(model.t, model.n, initialize = D_t_n)
model.Q_capacity_m_n = pyo.Param(model.m, model.m, initialize = Q_capacity)
model.C_t_m = pyo.Param(model.t, model.m, initialize = C_t_m)
model.C_t_k = pyo.Param(model.t, model.k, initialize = C_t_k)
model.R_up_t_n = pyo.Param(model.t, model.n, initialize = R_up_t_n)
model.R_down_t_n = pyo.Param(model.t, model.n, initialize = R_down_t_n)
model.Elkjel_n = pyo.Param(model.n, initialize = Elkjel_cap )
model.A = pyo.Param(model.s, initialize=A)
model.B = pyo.Param(model.s, initialize=B)

C_elkjel = 30

"-------- VARIABLES ----------------------"
model.x_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.x_t_n_k = pyo.Var(model.t, model.n, model.k, within = pyo.NonNegativeReals)
model.v_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.x_spill_t_n_j = pyo.Var(model.t, model.n, model.j, within = pyo.NonNegativeReals)
model.w_spill_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.w_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.elkjel = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.q_flow_t_m_n = pyo.Var(model.t, model.m, model.m, within = pyo.NonNegativeReals)
model.r_up_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.r_up_t_n_k = pyo.Var(model.t, model.n, model.k, within = pyo.NonNegativeReals)
model.r_down_t_n_i = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.r_down_t_n_k = pyo.Var(model.t, model.n, model.k, within = pyo.NonNegativeReals)
model.r_down_t_n_ror = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.r_down_t_n_elkjel = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.l1 = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.l2 = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.l3 = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.l4 = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.l5 = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.x_frac_t_n = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.v_frac_t_n = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.slack_up = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.slack_down = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)
model.slack_energy = pyo.Var(model.t, model.n, within = pyo.NonNegativeReals)

In [ ]:
""" -------  Defining constraints and objective function -------  """

# -------- Objective function -----------------
def OBJ(model):
    return sum(
        sum(
            sum(
                model.C_t_m[t, m] * model.q_flow_t_m_n[t, m, n]    # Cost for import from n to m
                - model.C_t_m[t, m] * model.q_flow_t_m_n[t, n, m]  # Cost for export from n to m
                for m in model.m
            )
        
            + model.C_t_k[t, 'Nuclear']* model.x_t_n_k[t, n, 'Nuclear']
            + model.C_t_k[t, 'Thermal'] * 2.22 * model.x_t_n_k[t, n, 'Thermal']
            
            + C_shed_l1 * model.l1[t,n] + C_shed_l2 * model.l2[t,n] + C_shed_l3 * model.l3[t,n] + C_shed_l4 * model.l4[t,n] + C_shed_l5 * model.l5[t,n]  
            
            - C_elkjel * model.elkjel[t,n]

            + model.C_t_k[t, 'Nuclear'] * 0.1 * (model.r_up_t_n_k[t, n, 'Nuclear'] - model.r_down_t_n_k[t, n, 'Nuclear'])
            + model.C_t_k[t, 'Thermal'] * 2.22 * 0.1 * (model.r_up_t_n_k[t, n, 'Thermal'] - model.r_down_t_n_k[t, n, 'Thermal'])

            + C_slack_up * model.slack_up[t,n] + C_slack_down * model.slack_down[t,n] + C_slack_energy * model.slack_energy[t,n]

            for n in model.n
        )
        for t in model.t
    )
model.OBJ = pyo.Objective(rule=OBJ, sense=pyo.minimize)


# ------   Energy balance  ---------- #
def energy_balance(model, t, n):
    return (
        model.x_t_n_i[t, n]
        + sum(model.x_t_n_k[t, n, k] for k in model.k)
        + sum(model.X_t_n_j[t, n, j] - model.x_spill_t_n_j[t, n, j] for j in model.j)
        + sum(model.q_flow_t_m_n[t, m, n] for m in model.m) # Import
        - sum(model.q_flow_t_m_n[t, n, m] for m in model.m) # Export
        + model.l1[t,n] + model.l2[t,n] + model.l3[t,n] + model.l4[t,n] + model.l5[t,n] 
        - model.elkjel[t,n]
        + model.slack_energy[t,n]
    ) == model.D_t_n[t, n]
model.energy_balance = pyo.Constraint(model.t, model.n, rule=energy_balance)


# ------ Reserve requirements constraints --------- Scenario-specific requirements are given to the model --------------#
def reserve_cap_up(model, t, n):
    return (model.r_up_t_n_i[t,n]  + 
        sum(model.r_up_t_n_k[t,n,k] for k in model.k) + model.slack_up[t,n] >= model.R_up_t_n[t,n]
    )
model.reserve_cap_up = pyo.Constraint(model.t, model.n, rule = reserve_cap_up)

def reserve_cap_down(model, t, n):
    return( model.r_down_t_n_i[t,n] + 
        sum(model.r_down_t_n_k[t,n,k] for k in model.k) 
        + model.r_down_t_n_ror[t,n] + model.r_down_t_n_elkjel[t,n] + model.slack_down[t,n] == model.R_down_t_n[t,n]
    )
model.reserve_cap_down = pyo.Constraint(model.t, model.n, rule = reserve_cap_down)


# ------ Max production capacity constraints ------------ #

def max_prod_i(model, t, n):
    return model.v_t_n_i[t,n] + model.r_up_t_n_i[t,n] <= model.X_cap_n_i[n]
model.max_prod_i = pyo.Constraint(model.t, model.n, rule=max_prod_i)


def max_prod_k(model, t, n, k):
    return model.x_t_n_k[t,n,k] + model.r_up_t_n_k[t,n,k] <= model.X_cap_n_k[n,k]
model.max_prod_k = pyo.Constraint(model.t, model.n, model.k, rule = max_prod_k)

def max_spill_j(model, t, n, j):
    return model.x_spill_t_n_j[t,n,j] <= model.X_t_n_j[t,n,j]
model.max_spill_j = pyo.Constraint(model.t, model.n, model.j, rule = max_spill_j)

def max_shed(model, t, n):
    return model.l1[t,n] + model.l2[t,n] + model.l3[t,n] + model.l4[t,n] + model.l5[t,n] <= model.D_t_n[t,n]
model.max_shed = pyo.Constraint(model.t, model.n, rule = max_shed)

def max_elkjel(model, t, n):
    return model.elkjel[t,n] + model.r_down_t_n_elkjel[t,n] <= model.Elkjel_n[n]
model.max_elkjel = pyo.Constraint(model.t, model.n, rule = max_elkjel)

def max_ror_down(model, t, n):
    return model.r_down_t_n_ror[t,n] <= model.X_t_n_j[t,n,'Run_of_river'] * 0.15
model.max_ror_down = pyo.Constraint(model.t, model.n, rule = max_ror_down)

# -------  Minimum production constraints ------------------
def min_prod_i(model, t, n):
    return model.r_down_t_n_i[t,n] <= model.x_t_n_i[t,n]
model.min_prod_i = pyo.Constraint(model.t, model.n, rule = min_prod_i)   

def min_prod_k(model, t, n, k):
    return model.r_down_t_n_k[t,n,k] <= model.x_t_n_k[t,n,k]
model.min_prod_k = pyo.Constraint(model.t, model.n, model.k, rule = min_prod_k)   


# ------- Load shedding constraints ------------------------------
def loadshed_max_l1(model, t, n):
    return model.l1[t,n] <= L_max_l1
model.loadshed_max_l1 = pyo.Constraint(model.t, model.n, rule = loadshed_max_l1)

def loadshed_max_l2(model, t, n):
    return model.l2[t,n] <= L_max_l2
model.loadshed_max_l2 = pyo.Constraint(model.t, model.n, rule = loadshed_max_l2)

def loadshed_max_l3(model, t, n):
    return model.l3[t,n] <= L_max_l3
model.loadshed_max_l3 = pyo.Constraint(model.t, model.n, rule = loadshed_max_l3)

def loadshed_max_l4(model, t, n):
    return model.l4[t,n] <= L_max_l4
model.loadshed_max_l4 = pyo.Constraint(model.t, model.n, rule = loadshed_max_l4)

def loadshed_max_l5(model, t, n):
    return model.l5[t,n] <= L_max_l5
model.loadshed_max_l5 = pyo.Constraint(model.t, model.n, rule = loadshed_max_l5)



# ------  Transfer limits constraint with scenario-sepcific transfer capacities for base case scenario -----------------------""
def flow_capacity(model, t, i, j):
    return model.q_flow_t_m_n[t,i,j] <= model.Q_capacity_m_n[i,j]                 
   
model.flow_capacity = pyo.Constraint(model.t, model.m, model.m, rule = flow_capacity)


# ---------  Reservoir balance constraints -------------------------
def reservoir_balance(model, t, n):
    if t == model.t.first():
        return model.w_t_n_i[t,n] == model.W_start_n_i[n] + model.W_inflow_t_n_i[t,n] - model.v_t_n_i[t,n] - model.w_spill_t_n_i[t,n] - 0.1*model.r_up_t_n_i[t,n] + 0.1*model.r_down_t_n_i[t,n]
    else:
        return model.w_t_n_i[t,n] == model.w_t_n_i[t-step,n] + model.W_inflow_t_n_i[t,n] - model.v_t_n_i[t,n] - model.w_spill_t_n_i[t,n] - 0.1*model.r_up_t_n_i[t,n] + 0.1*model.r_down_t_n_i[t,n]
        
model.reservoir_balance = pyo.Constraint(model.t, model.n, rule = reservoir_balance)

def reservoir_level_end(model, n):
    t_end = model.t.last()
    return model.w_t_n_i[t_end, n] == model.W_start_n_i[n]
model.reservoir_level_end = pyo.Constraint(model.n, rule=reservoir_level_end)

def reservoir_level_max(model, t, n):
    return model.w_t_n_i[t, n] <= model.W_cap_n_i[n]
model.reservoir_level_max = pyo.Constraint(model.t, model.n, rule = reservoir_level_max)


# ----------- Energy equivalent constraints --------------------------

def scale_x(model, t, n):
    return model.x_t_n_i[t,n] == model.x_frac_t_n[t,n] * model.X_cap_n_i[n]
model.scale_x = pyo.Constraint(model.t, model.n, rule  = scale_x)

def scale_v(model, t, n):
    return model.v_t_n_i[t,n] == model.v_frac_t_n[t,n] * model.X_cap_n_i[n]
model.scale_v = pyo.Constraint(model.t, model.n, rule = scale_v)

def SegmentHydro(model, t, n, s):
    return model.x_frac_t_n[t, n] <= model.A[s] + model.B[s] * model.v_frac_t_n[t, n] 
model.HydroSegmentConstr = pyo.Constraint(model.t, model.n, model.s, rule=SegmentHydro)

def MaxV(model, t, n):
    return  model.v_t_n_i[t, n] <= model.X_cap_n_i[n]
model.MaxV = pyo.Constraint(model.t, model.n, rule = MaxV)

In [ ]:
# Solver
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
with SolverFactory('gurobi', solver_io="python") as opt:
    results = opt.solve(model, load_solutions = True, tee = True)


In [ ]:
# -------- Results from variables -------------------

x_t_n_i_results = [(t, n, pyo.value(model.x_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_x_t_n_i = pd.DataFrame(list(x_t_n_i_results), columns=['Timestamp', 'Area', 'x_t_n_i_value']).set_index('Timestamp')

x_t_n_k_results = [(t, n, k, pyo.value(model.x_t_n_k[t, n, k])) for t in model.t for n in model.n for k in model.k]
df_results_x_t_n_k = pd.DataFrame(list(x_t_n_k_results), columns=['Timestamp', 'Area', 'Type', 'x_t_n_k_value']).set_index('Timestamp')

v_t_n_i_results = [(t, n, pyo.value(model.v_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_v_t_n_i = pd.DataFrame(list(v_t_n_i_results), columns=['Timestamp', 'Area', 'x_t_n_i_value']).set_index('Timestamp')

x_spill_t_n_j =  [(t, n, j, pyo.value(model.x_spill_t_n_j[t, n, j])) for t in model.t for n in model.n for j in model.j]
df_results_x_spill_t_n_j = pd.DataFrame(list(x_spill_t_n_j), columns=['Timestamp', 'Area', 'Type', 'x_spill_t_n_j']).set_index('Timestamp')

w_spill_t_n_i =  [(t, n, pyo.value(model.w_spill_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_w_spill_t_n_i = pd.DataFrame(list(w_spill_t_n_i), columns=['Timestamp', 'Area', 'w_spill_t_n_i']).set_index('Timestamp')

w_t_n_i =  [(t, n, pyo.value(model.w_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_w_t_n_i = pd.DataFrame(list(w_t_n_i), columns=['Timestamp', 'Area', 'w_t_n_i']).set_index('Timestamp')

loadshed_results = [(t, n, pyo.value(model.l1[t, n]), pyo.value(model.l2[t, n]), pyo.value(model.l3[t, n]), pyo.value(model.l4[t, n]), pyo.value(model.l5[t, n])) 
    for t in model.t for n in model.n]
df_results_loadshed = pd.DataFrame(loadshed_results, columns=['Timestamp', 'Area', 'L1', 'L2', 'L3', 'L4', 'L5']).set_index('Timestamp')


q_flow_t_m_n = [(t, i, j, pyo.value(model.q_flow_t_m_n[t, i, j])) for t in model.t for i in model.m for j in model.m]
df_results_q_flow_t_m_n = pd.DataFrame(q_flow_t_m_n, columns=['Timestamp', 'From_node', 'To_node', 'q_import_t_m_n']).set_index('Timestamp')

r_up_t_n_i = [(t, n, pyo.value(model.r_up_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_r_up_t_n_i = pd.DataFrame(list(r_up_t_n_i), columns=['Timestamp', 'Area', 'r_up_t_n_i']).set_index('Timestamp')


r_up_t_n_k = [(t, n, k, pyo.value(model.r_up_t_n_k[t, n, k])) for t in model.t for n in model.n for k in model.k]
df_results_r_up_t_n_k = pd.DataFrame(list(r_up_t_n_k), columns=['Timestamp', 'Area', 'Type', 'r_up_t_n_k']).set_index('Timestamp')

r_down_t_n_i = [(t, n, pyo.value(model.r_down_t_n_i[t, n])) for t in model.t for n in model.n]
df_results_r_down_t_n_i = pd.DataFrame(list(r_down_t_n_i), columns=['Timestamp', 'Area', 'r_down_t_n_i']).set_index('Timestamp')


r_down_t_n_k = [(t, n, k, pyo.value(model.r_down_t_n_k[t, n, k])) for t in model.t for n in model.n for k in model.k]
df_results_r_down_t_n_k = pd.DataFrame(list(r_down_t_n_k), columns=['Timestamp', 'Area', 'Type', 'r_down_t_n_k']).set_index('Timestamp')

r_down_t_n_ror = [(t, n, pyo.value(model.r_down_t_n_ror[t, n])) for t in model.t for n in model.n]
df_results_r_down_t_n_ror = pd.DataFrame(list(r_down_t_n_ror), columns=['Timestamp', 'Area', 'r_down_t_n_ror']).set_index('Timestamp')

r_down_t_n_elkjel = [(t, n, pyo.value(model.r_down_t_n_elkjel[t, n])) for t in model.t for n in model.n]
df_results_r_down_t_n_elkjel = pd.DataFrame(list(r_down_t_n_elkjel), columns=['Timestamp', 'Area', 'r_down_t_n_elkjel']).set_index('Timestamp')

elkjel =  [(t, n, pyo.value(model.elkjel[t, n])) for t in model.t for n in model.n]
df_results_elkjel = pd.DataFrame(list(elkjel), columns=['Timestamp', 'Area', 'elkjel']).set_index('Timestamp')

x_frac_results = [(t, n, pyo.value(model.x_frac_t_n[t, n])) for t in model.t for n in model.n]
df_results_x_frac_t_n = pd.DataFrame(list(x_frac_results), columns=['Timestamp', 'Area', 'x_frac_t_n _value']).set_index('Timestamp')

v_frac_results = [(t, n, pyo.value(model.v_frac_t_n[t, n])) for t in model.t for n in model.n]
df_results_v_frac_t_n = pd.DataFrame(list(v_frac_results), columns=['Timestamp', 'Area', 'v_frac_t_n _value']).set_index('Timestamp')

slack_up =  [(t, n, pyo.value(model.slack_up[t, n])) for t in model.t for n in model.n]
df_results_slack_up = pd.DataFrame(list(slack_up), columns=['Timestamp', 'Area', 'slack_up']).set_index('Timestamp')

slack_down =  [(t, n, pyo.value(model.slack_down[t, n])) for t in model.t for n in model.n]
df_results_slack_down = pd.DataFrame(list(slack_down), columns=['Timestamp', 'Area', 'slack_down']).set_index('Timestamp')

slack_energy =  [(t, n, pyo.value(model.slack_energy[t, n])) for t in model.t for n in model.n]
df_results_slack_energy = pd.DataFrame(list(slack_energy), columns=['Timestamp', 'Area', 'slack_energy']).set_index('Timestamp')


In [ ]:
# ------------ Results from dual values ------------------


up_reg_prices = [(t,n, model.dual[model.reserve_cap_up[t,n]]) for t in model.t for n in model.n]
df_results_up_reg_prices = pd.DataFrame(list(up_reg_prices), columns=['Timestamp', 'Area', 'up_reg_prices']).set_index('Timestamp')

down_reg_prices = [(t,n, model.dual[model.reserve_cap_down[t,n]]) for t in model.t for n in model.n]
df_results_down_reg_prices = pd.DataFrame(list(down_reg_prices), columns=['Timestamp', 'Area', 'down_reg_prices']).set_index('Timestamp')

dual_reservoir_balance = [(t,n, model.dual[model.reservoir_balance[t,n]]) for t in model.t for n in model.n]
df_results_dual_reservoir_balance = pd.DataFrame(list(dual_reservoir_balance), columns=['Timestamp', 'Area', 'Dual reservoir']).set_index('Timestamp')

lambda_price = [(t,n, model.dual[model.energy_balance[t,n]]) for t in model.t for n in model.n]
df_results_lambda = pd.DataFrame(list(lambda_price), columns=['Timestamp', 'Area', 'Lambda']).set_index('Timestamp')

dual_max_prod_i = [(t,n, model.dual[model.max_prod_i[t,n]]) for t in model.t for n in model.n]
df_results_dual_maxprodi = pd.DataFrame(list(dual_max_prod_i), columns=['Timestamp', 'Area', 'dual_x_i']).set_index('Timestamp')

dual_flow = [(t,m,n, model.dual[model.flow_capacity[t,m,n]]) for t in model.t for m in model.m for n in model.n if (m, n) in Q_capacity]
df_results_dual_flow = pd.DataFrame(list(dual_flow), columns=['Timestamp', 'From_node', 'To_node', 'dual_flow']).set_index('Timestamp')




In [ ]:
# ------- Create csv files for variabel and dual values results -------------

def save_result_dataframes(output_folder):
    for var_name, df in globals().items():
        if isinstance(df, pd.DataFrame) and var_name.startswith("df_results_"):
            file_path = os.path.join(output_folder, f"{var_name}_2050.csv")
            df.to_csv(file_path)
            print(f"Lagret {var_name} til {file_path}")


save_result_dataframes(Path("Results_2050_BaseCase"))